In [1]:
pip install karateclub

Note: you may need to restart the kernel to use updated packages.


In [2]:
from karateclub import Graph2Vec

c:\Users\lenaa\Anaconda3\envs\MLNS_project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\lenaa\Anaconda3\envs\MLNS_project\lib\site-packages\pygsp\filters\simpletight.py:79: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if kerneltype is 'sf':
c:\Users\lenaa\Anaconda3\envs\MLNS_project\lib\site-packages\pygsp\filters\simpletight.py:82: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif kerneltype is 'wavelet':


In [4]:
import networkx as nx

In [7]:
import pickle

In [22]:
dataset = nx.read_gpickle('../data/HIV_networkx.pkl')

In [23]:
with open('../data/labels_networkx.pkl', 'rb') as f:
    labels = pickle.load(f)

In [62]:

dic_features = dict()
dic_elem = nx.get_node_attributes(g, "element")
dic_aromatic = nx.get_node_attributes(g, "aromatic")
dic_hcount = nx.get_node_attributes(g, "hcount")
dic_charge = nx.get_node_attributes(g, "charge")


for node in dic_features :
    dic_features[node]=[dic_elem[node]]
    dic_features[node].append(dic_aromatic[node])
    dic_features[node].append(dic_hcount[node])
    dic_features[node].append(dic_charge[node])



In [68]:
for g in dataset :
    dic_features = dict()
    dic_elem = nx.get_node_attributes(g, "element")
    dic_aromatic = nx.get_node_attributes(g, "aromatic")
    dic_hcount = nx.get_node_attributes(g, "hcount")
    dic_charge = nx.get_node_attributes(g, "charge")


    for node in dic_elem :
        dic_features[node]=[dic_elem[node]]
        dic_features[node].append(dic_aromatic[node])
        dic_features[node].append(dic_hcount[node])
        dic_features[node].append(dic_charge[node])
    nx.set_node_attributes(g,dic_features, 'feature')

element: str. This describes the element of the atom. Defaults to '*' meaning unknown.
aromatic: bool. Whether the atom is part of an (anti)-aromatic system. Defaults to False.
isotope: float. The mass of the atom. Defaults to unknown.
hcount: int. The number of implicit hydrogens attached to this atom. Defaults to 0.
charge: int. The charge of this atom. Defaults to 0.
class: int. The "class" of this atom. Defaults to 0.

In [69]:
nx.get_node_attributes(dataset[1], "feature")

{0: ['N', False, 2, 0],
 1: ['N', False, 1, 0],
 2: ['P', False, 0, 0],
 3: ['S', False, 0, 0],
 4: ['N', False, 1, 0],
 5: ['N', False, 2, 0],
 6: ['C', True, 0, 0],
 7: ['C', True, 1, 0],
 8: ['C', True, 1, 0],
 9: ['C', True, 1, 0],
 10: ['C', True, 1, 0],
 11: ['C', True, 1, 0]}

In [70]:
model = Graph2Vec(attributed=True, dimensions= 1228)

model.fit(dataset)
graph_emb = model.get_embedding()

In [44]:
graph_emb

array([[ 0.12991002,  0.04800719,  0.00109923, ..., -0.03025101,
        -0.03591761, -0.068644  ],
       [ 0.11026119,  0.03690032, -0.00518586, ..., -0.02304455,
        -0.03344508, -0.0585653 ],
       [ 0.13973689,  0.0453981 , -0.00557501, ..., -0.03539269,
        -0.03980611, -0.07669869],
       ...,
       [ 0.14303206,  0.05148113, -0.00523456, ..., -0.03294419,
        -0.04502172, -0.07931035],
       [ 0.14717014,  0.05364874, -0.00590712, ..., -0.03514357,
        -0.04195713, -0.07584204],
       [ 0.13481843,  0.04649133, -0.00147134, ..., -0.03525974,
        -0.04052421, -0.07656855]], dtype=float32)

In [12]:
pip install xgboost


     -------------------------------------- 70.9/70.9 MB 930.9 kB/s eta 0:00:00


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from time import time
from tqdm import tqdm
from xgboost import XGBClassifier

In [87]:
list(dict(dataset[1].degree()).values())

[3, 4, 6, 3, 4, 3, 5, 4, 4, 4, 4, 4]

In [91]:
import numpy as np

In [92]:
def create_features(g,emb):
    degree = list((dict(g.degree()).values()))
    num_edges = g.number_of_edges()
    num_nodes = g.number_of_nodes()
    laplacian = nx.laplacian_matrix(g).astype(np.float32).toarray()
    eigenvals, eigenvecs = np.linalg.eigh(laplacian)
    return emb+ [ np.mean(degree), np.max(degree)/len(degree), 100*num_edges / (num_nodes * (num_nodes - 1))] + list(eigenvals[-3:])

In [94]:
new_dataset = list()
for i in tqdm(range(len(dataset))):
    g = dataset[i]
    emb = list(graph_emb[i])
    features = create_features(g,emb)
    new_dataset.append(features)

100%|██████████| 2443/2443 [00:10<00:00, 228.43it/s]


In [95]:
train_dataset, test_dataset, train_labels, test_labels = train_test_split(graph_emb, labels, test_size=0.25, random_state=42)

In [96]:
xgb =XGBClassifier(random_state=0)

xgb.fit(train_dataset, train_labels)
predxgb = xgb.predict(test_dataset)

In [97]:
accuracy_score(test_labels, predxgb)

0.5597381342062193

In [18]:
import sklearn

In [98]:
sklearn.metrics.confusion_matrix(test_labels, predxgb)

array([[172,  66,   0],
       [126, 125,  21],
       [ 21,  35,  45]], dtype=int64)

In [99]:
from sklearn.cluster import KMeans

In [100]:
kmeans = KMeans(n_clusters=3, random_state=0, n_init="auto").fit(train_dataset)

In [101]:
pred =kmeans.labels_

In [102]:
accuracy_score(train_labels, pred)

0.37445414847161573